In [ ]:
import tensorflow as tf  
from keras.preprocessing.image import ImageDataGenerator  
from keras.regularizers import l2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

In [ ]:
tf.__version__  

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,  
                                   shear_range = 0.2,  
                                   zoom_range = 0.2,  
                                   horizontal_flip = True)  



In [ ]:
training_set = train_datagen.flow_from_directory('../input/awsazuregcplogo-dataset/dataset/training_set',  
                                                 target_size = (64, 64),  
                                                 batch_size = 32,  
                                                 class_mode = 'categorical')  

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)  


In [ ]:
test_set = test_datagen.flow_from_directory('../input/awsazuregcplogo-dataset/dataset/testing_set',  
                                            target_size = (64, 64),  
                                            batch_size = 32,  
                                            class_mode = 'categorical')  

In [ ]:
model = tf.keras.models.Sequential()  

In [ ]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))  


In [ ]:
l2_reg = 0

# 1st Conv Layer : CRP 
model.add(Conv2D(96, (11, 11), input_shape=(64,64,3),
    padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Conv Layer 
model.add(Conv2D(256, (5, 5),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024,(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 5th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024,  (3 , 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

# 1st FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC Layer
model.add(Dense(3))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# print(model.summary())

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x = training_set, validation_data = test_set, epochs = 75)

In [ ]:
model.save('./awsazuregcp.h5')

In [ ]:
from keras.models import load_model
mod = load_model("awsazuregcp.h5")

In [ ]:
from keras.preprocessing import image
img = image.load_img("../input/imgazure/1200px-Amazon_Web_Services_Logo.svg.png")

In [ ]:
img = image.img_to_array(img)

In [ ]:
import numpy as np
x = np.resize(img, (64,64,3))

In [ ]:
x.shape

In [ ]:
import numpy 
img_exp = numpy.expand_dims(x, axis=0)

In [ ]:
img_exp.shape

In [ ]:
mod.predict(img_exp)